In [3]:
from nltk import word_tokenize
import gensim.downloader as api
import torch

import plotly.graph_objects as go
import plotly.express as px

In [4]:
model = api.load("glove-twitter-25")

In [5]:
sentence = "Your journey starts with one step"
tokens = word_tokenize(sentence.lower())
tokens

['your', 'journey', 'starts', 'with', 'one', 'step']

In [6]:
embeddings = torch.tensor(model[tokens])
embeddings.shape

torch.Size([6, 25])

In [21]:
# attention_scores = torch.zeros(6,6)

# for i in range(len(embeddings)):
#     for j in range(len(embeddings)):
#         attention_scores[i,j] = torch.dot(embeddings[i],embeddings[j])

# same as the above code
attention_scores = embeddings @ embeddings.T


attention_weights = torch.softmax(attention_scores, dim=1)
attention_weights
# context vectors will be attention weights times the query

tensor([[9.9926e-01, 2.0778e-11, 2.7000e-10, 7.2412e-04, 1.7945e-05, 5.8528e-10],
        [7.5704e-01, 1.0926e-01, 3.8048e-03, 1.2260e-01, 7.1108e-03, 1.8562e-04],
        [1.1012e-01, 4.2592e-05, 8.6161e-01, 2.5294e-02, 2.9102e-03, 2.1668e-05],
        [8.1727e-01, 3.7977e-09, 6.9993e-08, 1.8245e-01, 2.8162e-04, 3.0426e-08],
        [9.4683e-01, 1.0297e-08, 3.7647e-07, 1.3165e-02, 4.0008e-02, 1.2469e-07],
        [9.5207e-01, 8.2877e-06, 8.6421e-05, 4.3853e-02, 3.8442e-03, 1.3385e-04]])

In [22]:
context_vectors = torch.matmul(attention_weights,embeddings)
context_vectors.shape

torch.Size([6, 25])

In [23]:
from_ = 2

x_points,y_points,z_points = embeddings[:, from_:from_+3].T
xc_points,yc_points,zc_points = context_vectors[:, from_:from_+3].T

points_trace = go.Scatter3d(
    x=x_points,
    y=y_points,
    z=z_points,
    mode='markers+text',
    marker={
        'size': 8,
        'opacity': 0.8,
        'color': "blue",
    },
    text=tokens
)

context_trace = go.Scatter3d(
    x=xc_points,
    y=yc_points,
    z=zc_points,
    mode='markers+text',
    marker={
        'size': 8,
        'opacity': 0.8,
        'color': "red",
    },
    text=tokens
)

layout = go.Layout( template='plotly_dark',
                    title="Embedding Vector",
                    margin = {"pad":0, "l":0, "r":0, "t":30, "b":10}
                   )
arrow_traces = []
for i in range(len(embeddings)):
    arrow_traces.append(go.Scatter3d(
        x=[0, x_points[i]],
        y=[0, y_points[i]],
        z=[0, z_points[i]],
        mode='lines',
        showlegend=False,
        marker=dict(color = "blue")
    ))

fig = go.Figure([points_trace,context_trace] + arrow_traces, layout=layout)

fig.show()


In [24]:

fig = px.imshow(attention_weights,
                labels = dict(color="Similarity"),
                x = tokens,
                y = tokens)
fig.show()